In [1]:
import pandas as pd
import numpy as np

In [25]:
LOW = 0.08
HIGH = 0.85

In [26]:
#load prediction file for both detection and classification
det = pd.read_csv('../checkpoint/1903/vfnet101/submission_22.csv')
cls = pd.read_csv('../checkpoint/1703/classifier/default.csv')

In [27]:
print(det.shape)
det.head()

(3000, 2)


,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,3 0.54 831 1222 1904 1649 11 0.51 1334 282 165...
1,004f33259ee4aef671c2b95d54e4be68,0 0.63 1246 573 1537 952 0 0.49 1092 572 1535 ...
2,008bdde2af2462e86fd373a445d0f4cd,0 0.83 1406 824 1745 1223 0 0.34 1471 823 1726...
3,009bc039326338823ca3aa84381f17f1,0 0.44 976 456 1226 744 3 0.85 662 1025 1564 1...
4,00a2145de1886cb9eb88869c85d74080,0 0.64 1123 709 1352 937 0 0.17 1174 708 1350 ...


In [28]:
print(cls.shape)
cls.head()

(3000, 2)


,image_id,target
0,002a34c58c5b758217ed1f584ccbcfe9,0.013326
1,004f33259ee4aef671c2b95d54e4be68,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0.939700
3,009bc039326338823ca3aa84381f17f1,0.123799
4,00a2145de1886cb9eb88869c85d74080,0.654006


In [29]:
final_pred = pd.merge(det, cls, on = 'image_id', how = 'left')
print(final_pred.shape)
final_pred.head()

(3000, 3)


,image_id,PredictionString,target
0,002a34c58c5b758217ed1f584ccbcfe9,3 0.54 831 1222 1904 1649 11 0.51 1334 282 165...,0.013326
1,004f33259ee4aef671c2b95d54e4be68,0 0.63 1246 573 1537 952 0 0.49 1092 572 1535 ...,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0 0.83 1406 824 1745 1223 0 0.34 1471 823 1726...,0.939700
3,009bc039326338823ca3aa84381f17f1,0 0.44 976 456 1226 744 3 0.85 662 1025 1564 1...,0.123799
4,00a2145de1886cb9eb88869c85d74080,0 0.64 1123 709 1352 937 0 0.17 1174 708 1350 ...,0.654006


In [30]:
#Number of 'No Findings' before filtering
final_pred['PredictionString'].value_counts().iloc[[0]]

14 1 0 0 1 1    20
Name: PredictionString, dtype: int64

In [31]:
def filter_class(row, low_thr=LOW, high_thr=HIGH):
    prob = row['target']
    if prob<low_thr:
        ## Less chance of having any disease
        row['PredictionString'] = '14 1 0 0 1 1'
    elif low_thr<=prob<high_thr:
        ## More change of having any diesease
        row['PredictionString']+=f' 14 {prob} 0 0 1 1'
    elif high_thr<=prob:
        ## Good chance of having any disease so believe in object detection model
        row['PredictionString'] = row['PredictionString']
    else:
        raise ValueError('Prediction must be from [0-1]')
    return row

In [32]:
submission = final_pred.apply(filter_class, axis=1)
submission.head()

,image_id,PredictionString,target
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1,0.013326
1,004f33259ee4aef671c2b95d54e4be68,14 1 0 0 1 1,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0 0.83 1406 824 1745 1223 0 0.34 1471 823 1726...,0.939700
3,009bc039326338823ca3aa84381f17f1,0 0.44 976 456 1226 744 3 0.85 662 1025 1564 1...,0.123799
4,00a2145de1886cb9eb88869c85d74080,0 0.64 1123 709 1352 937 0 0.17 1174 708 1350 ...,0.654006


In [33]:
#Number of 'No Findings' after filtering
submission['PredictionString'].value_counts().iloc[[0]]

14 1 0 0 1 1    1912
Name: PredictionString, dtype: int64

In [34]:
submission[['image_id', 'PredictionString']].to_csv('../checkpoint/submission.csv',index = False)